In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from enum import Enum
import random

class Colour(Enum):
    RED = 1
    ORANGE = 2
    YELLOW = 3
    GREEN = 4
    BLUE = 5
    INDIGO = 6
    VIOLET = 7
    WHITE = 8

def encode_colour_sequence(colours):
    """ convert a list of Colour enums to 1-based integer indices """
    return [c.value for c in colours]

def decode_colour_sequence(indices):
    """ convert a list of 1-based indices to Colour enums """
    return [Colour(i) for i in indices]

# Choose n random colours from the first c Colours (with possible repeats)
def random_secret_colours(n_positions, n_colours):
    """ generate random list of Colour enums of length n """
    return random.choices(list(Colour)[:n_colours], k=n_positions)

def decode_solution(memberships, k, n):
    """ decode secret code from measured membership bitstrings """
    solution = []
    for pos in range(n):
        bits = [int(memberships[(1, ci)][pos]) for ci in range(2, k+1)]
        if all(bits):
            solution.append(1)
        else:
            for i, bit in enumerate(bits, start=2):
                if bit:
                    solution.append(i)
                    break                            
    return decode_colour_sequence(solution)

def mastermind_deutsch_oracle(secret, c1, c2, n, log=False):    
    """ create oracle marking states where positions match c1 or c2 in the secret """
    qc = QuantumCircuit(n)
    for j in range(n):
        if secret[j] == c1:
            qc.x(j)
            qc.z(j)
            qc.x(j)
        if secret[j] == c2:
            qc.z(j)
    if log:
        print("Oracle depth:", qc.decompose().depth())
        print("Oracle size (gate count):", qc.size())
        print("Oracle width (qubits used):", qc.width())
    return qc

def find_two_colour_position(n, color_oracle, log=False):
    """ create and run Deutsch-Jozsa circuit to return probable bitstrings """
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    qc.compose(color_oracle, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))

    backend = AerSimulator()
    if log:
        print(f"Backend selected: {backend.name} with {backend.num_qubits} qubits")

    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    isa_circuit = pm.run(qc)
    
    if log:
        print("ISA circuit depth:", isa_circuit.decompose().depth())
        print("ISA circuit size (gate count):", isa_circuit.size())
        print("ISA circuit width (qubits used):", isa_circuit.width())
        print("------")
        display(isa_circuit.draw())
        # display(isa_circuit.draw("mpl"))
        
    with Session(backend=backend) as session:
        sampler = Sampler(mode=session)
        results = sampler.run([isa_circuit]).result()
        counts = results[0].data.c.get_counts()
    return max(counts, key=counts.get)

def mastermind_non_adaptive(k, n, secret_code, log=False):
    """ a non-adaptive algorithm iterating on the Deutsch-Jozsa-ish oracle """
    secret = encode_colour_sequence(secret_code)
    measurements = {}
    c1 = 1  # 1-based index for colours
    for ci in range(2, k + 1):
        oracle = mastermind_deutsch_oracle(secret, c1, ci, n, log=log)
        membership = find_two_colour_position(n, oracle, log=log)
        measurements[(c1, ci)] = membership[::-1]

    if log: 
        print("Quantum measured membership bitstrings for (c1, ci) pairs:")
        for ci in range(2, k + 1):
            print(f"(c1={c1}, ci={ci}): {measurements[(c1, ci)]}")

        print(f"\nClassical membership matrix (rows: positions, cols: colours 1-{k}):")
        for pos in range(n):
            print([int(secret[pos] == c) for c in range(1, k+1)])

    return decode_solution(measurements, k, n)


k = 6  # number of colours in game
n = 4  # number of positions in solution

# secret_colours = [Colour.BLUE, Colour.ORANGE, Colour.YELLOW, Colour.ORANGE]  # List of Colour enums
secret_code = random_secret_colours(n, k)

solution_colours = mastermind_non_adaptive(k, n, secret_code, log=True)

print("\nSecret code:", [c.name for c in secret_code])
print("Solution   :", [c.name for c in solution_colours])


Oracle depth: 1
Oracle size (gate count): 2
Oracle width (qubits used): 4
Backend selected: aer_simulator with 30 qubits
ISA circuit depth: 2
ISA circuit size (gate count): 6
ISA circuit width (qubits used): 8
------


┌─┐         
q_0: ──────────────┤M├─────────
                   └╥┘┌─┐      
q_1: ───────────────╫─┤M├──────
     ┌────────────┐ ║ └╥┘┌─┐   
q_2: ┤ U3(π,-π,0) ├─╫──╫─┤M├───
     ├────────────┤ ║  ║ └╥┘┌─┐
q_3: ┤ U3(π,-π,0) ├─╫──╫──╫─┤M├
     └────────────┘ ║  ║  ║ └╥┘
c: 4/═══════════════╩══╩══╩══╩═
                    0  1  2  3

Oracle depth: 1
Oracle size (gate count): 2
Oracle width (qubits used): 4
Backend selected: aer_simulator with 30 qubits
ISA circuit depth: 2
ISA circuit size (gate count): 6
ISA circuit width (qubits used): 8
------


┌────────────┐   ┌─┐   
q_0: ┤ U3(π,-π,0) ├───┤M├───
     ├────────────┤   └╥┘┌─┐
q_1: ┤ U3(π,-π,0) ├────╫─┤M├
     └────┬─┬─────┘    ║ └╥┘
q_2: ─────┤M├──────────╫──╫─
          └╥┘      ┌─┐ ║  ║ 
q_3: ──────╫───────┤M├─╫──╫─
           ║       └╥┘ ║  ║ 
c: 4/══════╩════════╩══╩══╩═
           2        3  0  1

Oracle depth: 0
Oracle size (gate count): 0
Oracle width (qubits used): 4
Backend selected: aer_simulator with 30 qubits
ISA circuit depth: 1
ISA circuit size (gate count): 4
ISA circuit width (qubits used): 8
------


...

Oracle depth: 0
Oracle size (gate count): 0
Oracle width (qubits used): 4
Backend selected: aer_simulator with 30 qubits
ISA circuit depth: 1
ISA circuit size (gate count): 4
ISA circuit width (qubits used): 8
------


┌─┐         
q_0: ┤M├─────────
     └╥┘┌─┐      
q_1: ─╫─┤M├──────
      ║ └╥┘┌─┐   
q_2: ─╫──╫─┤M├───
      ║  ║ └╥┘┌─┐
q_3: ─╫──╫──╫─┤M├
      ║  ║  ║ └╥┘
c: 4/═╩══╩══╩══╩═
      0  1  2  3

Oracle depth: 0
Oracle size (gate count): 0
Oracle width (qubits used): 4
Backend selected: aer_simulator with 30 qubits
ISA circuit depth: 1
ISA circuit size (gate count): 4
ISA circuit width (qubits used): 8
------


┌─┐         
q_0: ┤M├─────────
     └╥┘┌─┐      
q_1: ─╫─┤M├──────
      ║ └╥┘┌─┐   
q_2: ─╫──╫─┤M├───
      ║  ║ └╥┘┌─┐
q_3: ─╫──╫──╫─┤M├
      ║  ║  ║ └╥┘
c: 4/═╩══╩══╩══╩═
      0  1  2  3

Quantum measured membership bitstrings for (c1, ci) pairs:
(c1=1, ci=2): 0011
(c1=1, ci=3): 1100
(c1=1, ci=4): 0000
(c1=1, ci=5): 0000
(c1=1, ci=6): 0000

Classical membership matrix (rows: positions, cols: colours 1-6):
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]

Secret code: ['YELLOW', 'YELLOW', 'ORANGE', 'ORANGE']
Solution   : ['YELLOW', 'YELLOW', 'ORANGE', 'ORANGE']
